**O objetivo do exercício é implementar o ReAct.**

* Testar no dataset do IIRC - 50-100 primeiras perguntas com resposta (excluir as None)
* Usar o LLaMA (via groq ou ollama).
* Instruir o modelo a seguir a sequência Thougth, Action, Input, Observation (a observação não é do próprio modelo, mas resultado da busca)
* Instruir o modelo agir passo-a-passo (decomposição da pergunta, etc.).
* Usar a busca como ferramenta
* Usar o mesmo buscador do exercício de RAG
* Pensar em outras ferramentas (ex.: calculadora para perguntas aritméticas)

In [1]:
# Instala as bibliotecas necessárias com a opção '-qU', que garante uma instalação silenciosa e atualizada.
!pip install -qU langchain sentence_transformers groq  # Instala as bibliotecas langchain, sentence_transformers e groq.
!pip install -qU langchain-huggingface langchain-community  # Instala bibliotecas para integração com HuggingFace e a comunidade LangChain.
!pip install -qU faiss-cpu py_expression_eval  # Instala o FAISS para indexação de vetores (versão CPU) e py_expression_eval para avaliação de expressões matemáticas.
!pip install -qU langchain-text-splitter  # Instala a biblioteca para divisão de textos da LangChain.

# Baixa o arquivo de teste do dataset IIRC (situado em um servidor na Amazon S3).
!wget https://iirc-dataset.s3.us-west-2.amazonaws.com/iirc_test.json

# Baixa um arquivo compactado (.tar.gz) que contém artigos de contexto, também hospedado no servidor S3 da Amazon.
!wget https://iirc-dataset.s3.us-west-2.amazonaws.com/context_articles.tar.gz

# Extrai o conteúdo do arquivo compactado baixado (context_articles.tar.gz).
!tar -xf context_articles.tar.gz


ERROR: Could not find a version that satisfies the requirement langchain-text-splitter (from versions: none)
ERROR: No matching distribution found for langchain-text-splitter
--2024-10-09 21:28:40--  https://iirc-dataset.s3.us-west-2.amazonaws.com/iirc_test.json
Resolving iirc-dataset.s3.us-west-2.amazonaws.com (iirc-dataset.s3.us-west-2.amazonaws.com)... 3.5.79.133, 52.218.217.65, 3.5.83.233, ...
Connecting to iirc-dataset.s3.us-west-2.amazonaws.com (iirc-dataset.s3.us-west-2.amazonaws.com)|3.5.79.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2874825 (2.7M) [application/json]
Saving to: ‘iirc_test.json.2’

iirc_test.json.2    100%[===================>]   2.74M   739KB/s    in 3.8s    

2024-10-09 21:28:44 (739 KB/s) - ‘iirc_test.json.2’ saved [2874825/2874825]

--2024-10-09 21:28:44--  https://iirc-dataset.s3.us-west-2.amazonaws.com/context_articles.tar.gz
Resolving iirc-dataset.s3.us-west-2.amazonaws.com (iirc-dataset.s3.us-west-2.amazonaws.com)... 52.

In [2]:
#@title Importação de bibliotecas necessárias
from langchain_text_splitters import RecursiveCharacterTextSplitter  # Divisor de texto recursivo da LangChain
from langchain_huggingface.embeddings import HuggingFaceEmbeddings  # Gera embeddings usando HuggingFace
from sklearn.metrics import precision_score, recall_score, f1_score  # Métricas de avaliação (precisão, recall, F1-score)
from py_expression_eval import Parser  # Avaliador de expressões matemáticas
from collections import Counter  # Contador para operações com coleções
from groq import Groq  # Cliente da API Groq
from tqdm import tqdm  # Barra de progresso
import numpy as np  # Biblioteca para operações numéricas
import json  # Manipulação de arquivos JSON
import faiss  # Ferramenta para indexação e busca de vetores
import re  # Biblioteca de expressões regulares

# Configuração da chave da API para Groq
API_KEY = "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"

# Inicialização do cliente Groq com a chave da API fornecida
groq_client = Groq(api_key=API_KEY)

# Exibe os métodos e atributos disponíveis no cliente Groq
print(dir(groq_client))  # Isso irá imprimir todos os métodos e atributos disponíveis


['__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_base_url', '_build_headers', '_build_request', '_calculate_retry_timeout', '_client', '_custom_headers', '_custom_query', '_default_stream_cls', '_enforce_trailing_slash', '_idempotency_header', '_idempotency_key', '_is_protocol', '_limits', '_make_sse_decoder', '_make_status_error', '_make_status_error_from_response', '_maybe_override_cast_to', '_parse_retry_after_header', '_platform', '_prepare_options', '_prepare_request', '_prepare_url', '_process_response', '_process_response_data', '_proxies', '_remaining_retries', '_request', 

In [3]:
#@title Carregamento dos dados a partir de arquivos JSON

# Carrega os artigos de contexto do arquivo 'context_articles.json' para a variável 'context_articles'
with open("context_articles.json", 'r') as file:
    context_articles = json.load(file)  # Armazena os dados carregados em 'context_articles'

# Carrega os dados de teste do arquivo 'iirc_test.json' para a variável 'data'
with open("iirc_test.json", 'r') as file:
    data = json.load(file)  # Armazena os dados carregados em 'data'


In [4]:
#@title Função para filtrar respostas do tipo 'none'
def exclude_none_answers(data):
    filtered_data = []  # Lista para armazenar os itens filtrados
    for item in data:
        filtered_questions = []  # Lista temporária para armazenar as perguntas filtradas
        for question in item['questions']:
            # Verifica se a resposta não é do tipo 'none'
            if question['answer']['type'] != 'none':
                question['title'] = item['title']  # Adiciona o título do item à pergunta
                filtered_questions.append(question)  # Adiciona a pergunta filtrada à lista
        if filtered_questions:  # Se houver perguntas filtradas
            filtered_item = item.copy()  # Faz uma cópia do item original
            filtered_item['questions'] = filtered_questions  # Substitui as perguntas pelo conjunto filtrado
            filtered_data.append(filtered_item)  # Adiciona o item filtrado à lista final
    return filtered_data  # Retorna a lista final com as perguntas filtradas

# Imprime o total de perguntas antes do filtro
print(f'Total questions: {len(data)}')

# Aplica o filtro para remover respostas do tipo 'none'
filter_data = exclude_none_answers(data)

# Imprime o total de perguntas após o filtro
print(f'Filter questions: {len(filter_data)}')

# Exibe uma amostra dos dados filtrados
print("\n===> Show one sample <===")
filter_data[0]


Total questions: 514
Filter questions: 435

===> Show one sample <===


{'questions': [{'answer': {'type': 'span',
    'answer_spans': [{'text': 'sky and thunder god',
      'passage': 'zeus',
      'type': 'answer',
      'start': 83,
      'end': 102}]},
   'question': 'What is Zeus know for in Greek mythology?',
   'context': [{'text': 'he Palici the sons of Zeus',
     'passage': 'main',
     'indices': [684, 710]},
    {'text': 'in Greek mythology', 'passage': 'main', 'indices': [137, 155]},
    {'text': 'Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion',
     'passage': 'Zeus',
     'indices': [0, 110]}],
   'question_links': ['Greek mythology', 'Zeus'],
   'title': 'Palici'}],
 'text': "The Palici (Παλικοί in Greek), or Palaci, were a pair of indigenous Sicilian chthonic deities in Roman mythology, and to a lesser extent in Greek mythology. They are mentioned in Ovid's Metamorphoses V, 406, and in Virgil's Aeneid IX, 585. Their cult centered on three small lakes that emitted sulphurous va

In [5]:
#@title Carregamento dos documentos
# Adaptado de Visconde https://github.com/neuralmind-ai/visconde

num_questions = 50  # Define o número de perguntas a serem consideradas

documents = []  # Lista para armazenar os documentos carregados
all_titles = []  # Lista para rastrear títulos já adicionados

# Itera sobre os itens filtrados (limitado pelo número de perguntas definido)
for item in filter_data[:num_questions]:
    # Verifica se o título do item (em minúsculas) ainda não foi adicionado
    if item['title'].lower() not in all_titles:
        # Adiciona o documento principal à lista de documentos
        documents.append({
            "title": item['title'],  # Título do documento
            "content": item["text"]   # Conteúdo do documento
        })
        all_titles.append(item['title'].lower())  # Armazena o título na lista de títulos já processados

    # Itera sobre os links do item para carregar documentos relacionados
    for link in item["links"]:
        # Verifica se o link do alvo está nos artigos de contexto e não foi adicionado ainda
        if link['target'].lower() in context_articles and link['target'].lower() not in all_titles:
            # Adiciona o documento relacionado ao artigo de contexto
            documents.append({
                "title": link['target'],  # Título do documento
                "content": context_articles[link['target'].lower()]  # Conteúdo do documento a partir dos artigos de contexto
            })
            all_titles.append(link['target'].lower())  # Armazena o título do link processado
        else:
            # Se o documento já foi carregado, exibe uma mensagem indicando isso
            print(f"Loaded doc: {link['target'].lower()}")

# Imprime o total de documentos carregados
print(f'Total documents: {len(documents)}')


Loaded doc: 9th paratroopers assault regiment "col moschin"
Loaded doc: goldfinger (film)
Loaded doc: list of international cricket council members
Loaded doc: icc americas championship
Loaded doc: the rev
Loaded doc: avenged sevenfold
Loaded doc: fox footy
Loaded doc: herald sun
Loaded doc: fox footy
Loaded doc: herald sun
Loaded doc: united states
Loaded doc: judeo-iraqi arabic
Loaded doc: maya civilization
Loaded doc: black watch
Loaded doc: suicidal tendencies
Loaded doc: western hockey league
Loaded doc: national hockey league
Loaded doc: home run
Loaded doc: minor league baseball
Loaded doc: colonel
Loaded doc: colonel
Loaded doc: israel
Loaded doc: harvard business review
Loaded doc: american football
Loaded doc: college football
Loaded doc: united states
Loaded doc: billboard 200
Loaded doc: romeo discography
Loaded doc: billboard 200
Loaded doc: master p
Loaded doc: hip hop history
Loaded doc: billboard 200
Loaded doc: louisiana
Loaded doc: arizona
Loaded doc: state farm stadi

In [6]:
#@title Divisão de texto em chunks (partes menores)

# Inicializa o divisor de texto com os parâmetros definidos
text_splitter = RecursiveCharacterTextSplitter(
    # Lista padrão de separadores usados para dividir o texto
    separators=["\n\n", "\n", " ", ""],  # Prioridade dos separadores: parágrafo, nova linha, espaço, e vazio
    chunk_size=1000,  # Tamanho máximo de cada chunk (parte)
    chunk_overlap=100,  # Quantidade de sobreposição entre os chunks
    length_function=len,  # Função utilizada para calcular o tamanho do chunk
    is_separator_regex=False,  # Define se os separadores são expressões regulares (nesse caso, não são)
)

# Cria uma lista com os conteúdos de cada documento
contents = [doc['content'] for doc in documents]

# Divisão (chunking) dos conteúdos em partes menores
chunks = []  # Lista para armazenar os chunks resultantes
for content in contents:
    # Divide o conteúdo e adiciona os chunks resultantes à lista
    chunks.extend(text_splitter.split_text(content))

# Exibe o total de chunks gerados
print(f'Total chunks: {len(chunks)}')

# Exibe o comprimento dos primeiros 10 chunks
print("\nLength of the first 10 chunks:")
print([len(text) for text in chunks][:10])

# Exibe uma amostra do primeiro chunk gerado
print("\n===> Show one sample <===")
print(chunks[0])


Total chunks: 33037

Length of the first 10 chunks:
[836, 992, 976, 948, 385, 596, 750, 739, 943, 775]

===> Show one sample <===
The Palici (Παλικοί in Greek), or Palaci, were a pair of indigenous Sicilian chthonic deities in Roman mythology, and to a lesser extent in Greek mythology. They are mentioned in Ovid's Metamorphoses V, 406, and in Virgil's Aeneid IX, 585. Their cult centered on three small lakes that emitted sulphurous vapors in the Palagonia plain, and as a result these twin brothers were associated with geysers and the underworld. There was also a shrine to the Palaci in Palacia, where people could subject themselves or others to tests of reliability through divine judgement; passing meant that an oath could be trusted. The mythological lineage of the Palici is uncertain; one legend made the Palici the sons of Zeus, or possibly Hephaestus, by Aetna or Thalia, but another claimed that the Palici were the sons of the Sicilian deity Adranus.


In [7]:
#@title Download do modelo para calcular embeddings

# Inicializa o modelo de embeddings utilizando HuggingFace (modelo pré-treinado específico)
model_emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Exemplo de texto para gerar embeddings
text = "This is a test document."  # Texto de entrada para teste
query_result = model_emb.embed_query(text)  # Gera o embedding para o texto fornecido

# Exibe o exemplo de geração de embeddings
print("\n\nExample to generate embeddings")
print(f'Text input: {text}')  # Exibe o texto de entrada
print(f'Embedding size: {len(query_result)}')  # Exibe o tamanho do embedding gerado
print(f'Embedding sample: {query_result[:3]}')  # Exibe uma amostra dos primeiros 3 valores do embedding


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavio



Example to generate embeddings
Text input: This is a test document.
Embedding size: 768
Embedding sample: [-0.04895172640681267, -0.039861924946308136, -0.021562771871685982]


In [8]:
#@title Cálculo dos embeddings para cada chunk (parte menor de texto)

# Criação de batches (lotes) usando list comprehension
batch_size = 128  # Define o tamanho de cada lote
# Divide os chunks em lotes de tamanho definido (batch_size)
batches = [chunks[i:i + batch_size] for i in range(0, len(chunks), batch_size)]

# Lista para armazenar os embeddings calculados
embeddings_list = []

# Itera sobre cada lote de chunks e calcula os embeddings
for chunk in tqdm(batches):  # Usa tqdm para visualizar o progresso da operação
    embedding = model_emb.embed_documents(chunk)  # Calcula os embeddings para os documentos no lote
    embeddings_list.extend(embedding)  # Adiciona os embeddings calculados à lista final

# Exibe o total de embeddings gerados
print(f'Total embeddings: {len(embeddings_list)}')

# Exibe a dimensão (tamanho) dos embeddings gerados
print(f"Embedding dimension: {len(embeddings_list[0])}")


100%|██████████| 259/259 [10:38<00:00,  2.46s/it]

Total embeddings: 33037
Embedding dimension: 768


In [9]:
#@title Criação do índice FAISS

# Converte a lista de embeddings para um array numpy
embeddings_array = np.array(embeddings_list)

# Define a dimensão dos embeddings com base no tamanho do primeiro vetor
dimension = len(embeddings_array[0])

# Cria o índice FAISS usando a métrica de distância L2 (distância euclidiana)
index = faiss.IndexFlatL2(dimension)

# Adiciona os embeddings ao índice FAISS
index.add(embeddings_array)

# Salva o índice em um arquivo
faiss.write_index(index, 'index_file.index')


In [10]:
#@title Carregamento do índice FAISS

# Lê o arquivo de índice FAISS previamente salvo
index = faiss.read_index('index_file.index')


In [11]:
#@title Ferramentas

# Função para recuperar chunks (partes menores de texto) relevantes com base em uma query
def retrieve_chunks(query, k, model, chunks):
    # Gera o embedding da query
    query_embedding = model.embed_query(query)

    # Converte o embedding da query para um array numpy com o formato correto
    query_embedding = np.array([query_embedding])

    # Realiza a busca pelos k vizinhos mais próximos no índice FAISS
    distances, indices = index.search(query_embedding, k)

    # Recupera os chunks correspondentes aos índices encontrados
    retrieved_chunks = [chunks[i] for i in indices[0]]

    return retrieved_chunks

# Calculadora para avaliar expressões matemáticas simples
parser = Parser()  # Inicializa o parser para a calculadora
def calculator(str):
    return parser.parse(str).evaluate({})  # Avalia a expressão passada como string

# Exemplo de consulta usando o embedding da query
query = "Your search query here"  # Exemplo de consulta de pesquisa
similar_docs = retrieve_chunks(query, k=1, model=model_emb, chunks=chunks)  # Recupera o chunk mais relevante

# Exibe o resultado da ferramenta de recuperação de chunks
print("Ferramenta de Teste: Chunks Recuperados:")
print(f"Consulta: {query}")
for i, doc in enumerate(similar_docs):
    print(f"[Documento {i+1}] : {doc}")
    print("-" * 50)

# Exemplo de uso da calculadora para avaliar uma expressão matemática
str_sum = "2 + 2 / 2"
print("\n\nFerramenta de Teste: Calculadora:")
print(f"Expressão matemática: {str_sum}")
print(f"Resultado: {calculator(str_sum)}")


Ferramenta de Teste: Chunks Recuperados:
Consulta: Your search query here
[Documento 1] : <a href="Zendesk">Zendesk</a>, <a href="Full%20Compass%20Systems">Full Compass Systems</a>, <a href="Raven%20Software">Raven Software</a>, and <a href="TDS%20Telecom">TDS Telecom</a>.
--------------------------------------------------


Ferramenta de Teste: Calculadora:
Expressão matemática: 2 + 2 / 2
Resultado: 3.0


In [21]:
#@title Definição do agente


System_prompt = """

Your task is to answer questions given the context information and not rely on prior knowledge. If you cannot answer the question, request assistance or use a tool.

You have access to the following tools:

Retrieval: useful when you need to answer questions about current events or information not provided in the context. The action input should be the relevant information about the question, possibly split into subquestions.

Calculator: useful when you need to calculate mathematical expressions. The action input is a string representing the mathematical expression, e.g., "2 + 2".

Response To Question: use this action when you are ready to provide the final answer to the question.

You will receive a question, then you should start a loop and do the following steps:

Thought: you should always think about what to do.
Action: the action to take, should be one of [Retrieval, Calculator, Response To Question].
Action Input: "the input to the action, to be sent to the tool."

After this, wait for the human to respond with an Observation, then you can continue.

If you have found the information needed to answer the question, you should use the following format:
Thought: you should always think about what to do.
Action: Response To Question
Response: "Return the concise, exact, and short answer to the question without any reasoning. If the answer is not clear, return 'None'. If applicable, provide the numerical value with its unit."

Begin!

"""


# Função para extrair a ação do texto
def extract_action(text):
    action_pattern = r'Action: (.*)'  # Padrão para capturar a ação
    action = re.search(action_pattern, text)
    if action:
        return action.group(1).strip().lower()  # Retorna a ação encontrada em minúsculas
    else:
        return None

# Função para extrair o input da ação
def extract_input(text):
    input_pattern = r'Action Input: "(.*?)"'  # Padrão para capturar o input da ação
    input = re.search(input_pattern, text)
    if input:
        return input.group(1).strip()  # Retorna o input encontrado
    else:
        return None

# Função para extrair a resposta do texto
def extract_response(text):
    observation_pattern = r'Response: (.*)'  # Padrão para capturar a resposta
    observation = re.search(observation_pattern, text)
    if observation:
        return observation.group(1).strip()  # Retorna a resposta encontrada
    else:
        return None

# Função para verificar se o texto é uma expressão matemática
def is_math_expr(text):
    """
    Usa uma expressão regular para verificar se o texto é uma
    expressão matemática entre dois números (ex: "10 + 5", "2 * 4").
    """
    pattern = r"^\d+(\s*[\+\-\*\/]\s*\d+)+$"
    match = re.match(pattern, text)
    return bool(match)

# Função principal do agente, que processa a consulta e utiliza ferramentas conforme necessário
def Stream_agent(query, max_steps=5, debug=False):
    messages = [
        { "role": "system", "content": System_prompt },  # Mensagem inicial do sistema
        { "role": "user", "content": query },  # Consulta do usuário
    ]

    # Etapa 1: recebe a mensagem do usuário
    if debug:
        print("\n\n")
        print("=" * 50)
        print(f"[DEBUG] Question: {query}")

    # Etapa 2: gera chamadas para as ferramentas (se necessário)
    steps = 0
    while True:
        response = client.chat.completions.create(
            messages=messages,
            model="llama3-70b-8192",  # Modelo utilizado
            temperature=0.1  # Controla a aleatoriedade da resposta
        )
        response_message = response.choices[0].message.content
        if debug:
            print(f"\n>> {response_message}")

        # Extrai a ação e o input da ação da resposta
        action = extract_action(response_message)
        action_input = extract_input(response_message)
        if debug:
            print(f"[DEBUG] Action: {action}")
            print(f"[DEBUG] Action Input: {action_input}")

        # Inicializa a observação
        observation = None
        if action == "retrieval":
            if action_input is None:
                # Se o input estiver vazio, usa a query como input
                action_input = query
            observation = ""
            tool_ans = retrieve_chunks(action_input, k=3, model=model_emb, chunks=chunks)  # Usa a ferramenta de busca
            for i, doc in enumerate(tool_ans):
                observation += f"[Document {i+1}] : {doc}\n"

        elif action == "calculator":
            observation = "The Action Input is not a valid math expression. Valid expression is: 2 + 2"
            if is_math_expr(action_input):  # Verifica se a expressão é matemática
                observation = calculator(action_input)

        elif action == "response to question":
            response = extract_response(response_message)  # Extrai a resposta final
            if debug:
                print(f"[DEBUG] Response: {response}")
            return response
            break

        else:
            observation = "Invalid action, should be one of [Retrieval, Calculator]"

        if debug:
            print(f"[DEBUG] Observation: {observation}")

        # Adiciona as mensagens ao histórico de conversas
        messages.extend([
            { "role": "system", "content": response_message },
            { "role": "user", "content": f"Observation: {observation}" },
        ])

        steps += 1
        if steps >= max_steps:
            if debug:
                print("Maximum number of steps reached. Stopping.")
            return None
            break

# Função para responder a múltiplas perguntas
def answer_questions(user_question, debug=False):
    results = {}
    results["Question"] = []
    results["Model response"] = []
    results["Ground truth"] = []

    n_questions = len(user_question)
    i = 1
    for item in user_question:
        print(f"Question: [{i}/{n_questions}]")
        i += 1
        query = item['questions'][0]['question']  # Extrai a pergunta

        # Executa o agente para obter a resposta
        response = Stream_agent(query, debug=debug)

        # Adaptação do Visconde https://github.com/neuralmind-ai/visconde
        hit = item['questions'][0]
        true_answer = ""
        if hit['answer']['type'] == 'span':
            true_answer = ", ".join([a['text'] for a in hit['answer']["answer_spans"]])
        elif hit['answer']['type'] == 'value':
            true_answer = "{0} {1}".format(hit['answer']['answer_value'], hit['answer']['answer_unit'])
        elif hit['answer']['type'] == "binary":
            true_answer = hit['answer']['answer_value']
        else:
            print("Answer type not supported")
            true_answer = ""

        results["Question"].append(query)
        results["Model response"].append(response)
        results["Ground truth"].append(true_answer)

    return results

# Inicializa o cliente da API Groq
client = Groq(
    api_key=API_KEY,
)


In [22]:
#@title Depuração do processo de resposta do agente e processo de resposta para um conjunto de perguntas

import random

# Seleciona 4 perguntas aleatórias de 'filter_data'
random_questions = random.sample(filter_data, 4)

# Adiciona a quarta pergunta (o quarto item de 'filter_data')
random_questions.append(filter_data[4])

# Função para dividir as perguntas em lotes menores e processá-las
def process_questions_in_batches(questions, batch_size=2):
    results = {"Question": [], "Ground truth": [], "Model response": []}
    for i in range(0, len(questions), batch_size):
        batch = questions[i:i + batch_size]
        batch_results = answer_questions(batch, debug=True)
        # Combina os resultados e filtra quaisquer entradas que contenham None
        combined_results = zip(
            batch_results["Question"],
            batch_results["Ground truth"],
            batch_results["Model response"]
        )
        filtered_results = [
            (q, gt, mr) for q, gt, mr in combined_results if None not in (q, gt, mr)
        ]
        for q, gt, mr in filtered_results:
            results["Question"].append(q)
            results["Ground truth"].append(gt)
            results["Model response"].append(mr)
    return results

# Processa as perguntas em lotes de tamanho 2 para evitar exceder o limite de contexto
results = process_questions_in_batches(random_questions, batch_size=2)

# Exibe o número de perguntas processadas
n = len(results["Question"])
print("\n\n===> Final response <===")

# Itera sobre todas as perguntas processadas e exibe as perguntas, respostas reais e as respostas do modelo
for i in range(n):
    print(f"Pergunta: {results['Question'][i]}")
    print(f"Resposta verdadeira: {results['Ground truth'][i]}")
    print(f"Resposta gerada pelo modelo: {results['Model response'][i]}")
    print("\n")  # Adiciona um espaço entre cada resposta para melhor visualização

num_questions = 50  # Define o número de perguntas a serem processadas
# Chama a função 'answer_questions' para processar as perguntas filtradas até o limite definido
results = answer_questions(filter_data[:num_questions])

# Remove entradas com None dos resultados
combined_results = zip(
    results["Question"],
    results["Ground truth"],
    results["Model response"]
)
filtered_results = [
    (q, gt, mr) for q, gt, mr in combined_results if None not in (q, gt, mr)
]
# Separa os resultados filtrados de volta nos respectivos campos
if filtered_results:
    results["Question"], results["Ground truth"], results["Model response"] = zip(*filtered_results)
else:
    results["Question"], results["Ground truth"], results["Model response"] = [], [], []

# Exibe o total de perguntas processadas
print(f'Total de perguntas: {len(results["Question"])}')


Question: [1/2]



[DEBUG] Question: Do Wowowin and Bubble Gang has any of the same cast members?

>> Thought: I need to find information about the cast members of Wowowin and Bubble Gang to determine if they have any common cast members.

Action: Retrieval
Action Input: "Cast members of Wowowin and Bubble Gang, Philippine TV shows"
[DEBUG] Action: retrieval
[DEBUG] Action Input: Cast members of Wowowin and Bubble Gang, Philippine TV shows
[DEBUG] Observation: [Document 1] : Founder.<ul><li>- <a href="Mike%20Quackenbush">Mike Quackenbush</a>
</li><li>- <a href="Reckless%20Youth">Reckless Youth</a>
</li></ul>
Owner.<ul><li>- <a href="Mike%20Quackenbush">Mike Quackenbush</a> (2002–present)
</li><li>- The Titor Conglomerate (storyline) (2010–2013)
</li><li>- <a href="Robbie%20Ellis">Robbie Ellis</a> (storyline) (2014–present)
</li></ul>
Commissioner.<ul><li>- <a href="Bob%20Saget">Bob Saget</a> (2006–2008)
</li><li>- <a href="Dave%20Coulier">Dave Coulier</a> (2008–2010)
</li></ul>
Directo

In [23]:
#@title Função para tokenizar o texto (converter em minúsculas e dividir em palavras)
def tokenize(text):
    return text.lower().split()

# Função para calcular o F1-Score usando a técnica de Bag of Words (BoW)
def bag_of_words_f1(reference, generated):
    # Verifica se a resposta verdadeira ou a resposta gerada são None
    if reference is None or generated is None:
        return 0

    # Tokeniza a referência e a resposta gerada
    ref_tokens = tokenize(reference)
    gen_tokens = tokenize(generated)

    # Converte os tokens para Counter (bag of words)
    ref_bag = Counter(ref_tokens)
    gen_bag = Counter(gen_tokens)

    # Calcula a interseção e a união dos conjuntos de tokens
    common_tokens = ref_bag & gen_bag
    correct = sum(common_tokens.values())

    # Verifica se uma das respostas está vazia
    if len(ref_bag) == 0 or len(gen_bag) == 0:
        # Se ambos estiverem vazios, o F1 é 1 se forem iguais, 0 caso contrário
        return int(ref_tokens == gen_tokens)

    # Se não houver tokens em comum, o F1 é 0
    if correct == 0:
        return 0

    # Calcula a precisão e o recall
    precision = 1.0 * correct / len(gen_tokens)
    recall = 1.0 * correct / len(ref_tokens)

    # Cálculo do F1-Score
    f1 = (2 * precision * recall) / (precision + recall)

    return f1

# Função para calcular a correspondência exata (Exact Match)
def exact_match(reference, generated):
    # Verifica se a resposta verdadeira ou a resposta gerada são None
    if reference is None or generated is None:
        return 0
    ref = reference.strip().lower()  # Remove espaços extras e converte para minúsculas
    gen = generated.strip().lower()

    return int(ref == gen)  # Retorna 1 se forem iguais, 0 caso contrário

# Exemplo de uso das funções
reference = "Ground truth"  # Resposta verdadeira
generated = "Example response: ground truth"  # Resposta gerada pelo modelo

# Calcula o F1-Score usando Bag of Words
f1 = bag_of_words_f1(reference, generated)
print("Exemplo de F1-BoW")
print(f"Resposta verdadeira: {reference}")
print(f"Saída do modelo: {generated}")
print(f"F1-Score: {f1:.4f}")  # Exibe o F1-Score com 4 casas decimais
print(f"Correspondência exata: {exact_match(reference, generated)}")  # Exibe o resultado de correspondência exata


Exemplo de F1-BoW
Resposta verdadeira: Ground truth
Saída do modelo: Example response: ground truth
F1-Score: 0.6667
Correspondência exata: 0


In [24]:
#@title Avaliação de performance do modelo

n = len(results["Question"])  # Número total de perguntas processadas
f1_score = []  # Lista para armazenar os valores do F1-Score para cada pergunta
exact_match_value = []  # Lista para armazenar os valores de correspondência exata (exact match) para cada pergunta

# Itera sobre cada pergunta e calcula o F1-Score e o valor de correspondência exata
for i in range(n):
    # Calcula o F1-Score usando Bag of Words e armazena o resultado
    f1_score.append(bag_of_words_f1(results["Ground truth"][i], results["Model response"][i]))
    # Calcula a correspondência exata e armazena o resultado
    exact_match_value.append(exact_match(results["Ground truth"][i], results["Model response"][i]))

# Calcula a taxa de correspondência exata (em porcentagem)
exact_rate = np.mean(exact_match_value) * 100

# Exibe os resultados de performance
print(f"Total de perguntas: {n}")  # Exibe o número total de perguntas
print(f"Média F1-Score : {np.mean(f1_score):.4f}")  # Exibe a média do F1-Score com 4 casas decimais
print(f"Taxa de correspondência: [{sum(exact_match_value)}/{n}], {exact_rate:.2f}%")  # Exibe a taxa de correspondência exata


Total de perguntas: 40
Média F1-Score : 0.2882
Taxa de correspondência: [8/40], 20.00%
